In [6]:
#!pip install dm-pybloom
#!pip install bitarray
#!pip install kafka-python
#!curl https://www.mit.edu/~ecprice/wordlist.10000 -o words.txt
#!ls -l
#!pip install --no-index ./../dm_pybloom-3.0.4.tar.gz
#!pip install --no-index ./../bitarray-2.7.3.tar.gz
#!pip install --no-index ./../kafka-python3-3.0.0.tar.gz

In [46]:
#GGlobal Variables
# Cenario 1
SERVER = 'PT'
TOPICOS ='Topicos'+SERVER
#topicos = ['Teatro', 'Musica', 'Series', 'Educacao', 'Teatro', 'Nutricao', 'Ginasio', 'ciencia', 'technologia']

all_words = list()
with open("words.txt",encoding='utf-8-sig') as f:
    for line in f:
        all_words.append(line.rstrip())

#shuffle(all_words) 
# first n element
topicos= all_words[249:749]
print(len(topicos))

In [49]:
#Global Variables
# Cenario 2
from random import shuffle, choice
SERVER = 'PT'
TOPICOS ='Topicos'+SERVER
#topicos = ['Teatro', 'Musica', 'Series', 'Educacao', 'Teatro', 'Nutricao', 'Ginasio', 'ciencia', 'technologia']

all_words = list()
with open("words.txt",encoding='utf-8-sig') as f:
    for line in f:
        all_words.append(line.rstrip())

shuffle(all_words) 
# first n element
topicos = all_words[:800]
print(len(topicos))

800


In [50]:
from kafka3 import TopicPartition, KafkaConsumer, KafkaProducer
from json import loads,dumps
from time import sleep
import hashlib
import numpy as np
from datetime import datetime

def subscribe(consumer, topico):
    lista = consumer.subscription()
    if lista is None:
        consumer.subscribe('Topicos'+SERVER)
    else:
        lista.add(topico)
        consumer.subscribe(lista)
        
dict_GW = {}
total_eventos = 0

consumer = KafkaConsumer( bootstrap_servers='kafka:9092', value_deserializer=lambda x: loads(x.decode('utf-8')))
producer = KafkaProducer(bootstrap_servers='kafka:9092' , value_serializer=lambda x:  dumps(x).encode('utf-8'))

print('Consumer '+ SERVER)

topicos_hash = list()
for x in topicos:
    topicos_hash.append(SERVER.lower()+'_'+hashlib.sha1(x.encode('utf-8')).hexdigest())  
    
#randonList = random.sample(range(1,len(topicos) ),4)
#for index in randonList:
for x in topicos_hash:
    subscribe(consumer,x) 

message = {'topico' : topicos_hash}
producer.send(TOPICOS,value = message)

producer.flush()
print ('Esperando por eventos ...')
for msg in consumer:
    total_eventos = total_eventos + 1
    if msg.value.get("from") is None:
        #mensagem local
        if dict_GW.get(SERVER) is None:
            dict_GW[SERVER] = []
        time_start = datetime.fromtimestamp(msg.value['time_start'])
        time_end = datetime.now()
        diff = time_end - time_start
        dict_GW[SERVER].append(diff)
    else:
        #mensagem local
        if dict_GW.get(msg.value['from']) is None:
            dict_GW[msg.value['from']] = []
        time_start = datetime.fromtimestamp(msg.value['message']['time_start'])
        time_end = datetime.now()
        diff = time_end - time_start
        dict_GW[msg.value['from']].append(diff)
    #print (msg.value)

Consumer PT
Esperando por eventos ...


KeyboardInterrupt: 

In [51]:
import datetime
print ('Consumer ' +SERVER)
print("Total Itens recebidos: {0}".format(total_eventos) )
for server in dict_GW:
    if SERVER != server:
        print("Total Itens roteados do {0}: {1}".format(server, len(dict_GW[server])))

for server in dict_GW:
    print("Tempo Medio do {0}: {1}".format(server,sum(dict_GW[server], datetime.timedelta())/len(dict_GW[server])))

Consumer PT
Total Itens recebidos: 49
Total Itens roteados do MZ: 7
Total Itens roteados do BR: 6
Tempo Medio do PT: 0:00:00.307725
Tempo Medio do MZ: 0:00:00.575195
Tempo Medio do BR: 0:00:00.582650


In [114]:
print(dict_GW)

{'MZ': [datetime.timedelta(microseconds=9393), datetime.timedelta(microseconds=5873), datetime.timedelta(microseconds=4817), datetime.timedelta(microseconds=5474), datetime.timedelta(microseconds=2683), datetime.timedelta(microseconds=5156), datetime.timedelta(microseconds=4978), datetime.timedelta(microseconds=4484), datetime.timedelta(microseconds=4512), datetime.timedelta(microseconds=17670), datetime.timedelta(microseconds=11125), datetime.timedelta(microseconds=11145), datetime.timedelta(microseconds=11219), datetime.timedelta(microseconds=7914), datetime.timedelta(microseconds=12444), datetime.timedelta(microseconds=11795), datetime.timedelta(microseconds=12372), datetime.timedelta(microseconds=11393)], 'BR': []}


In [41]:
e = [0,1,8,3,5,4,5,6,7,8,9,9,0,8,5,4,3,33,3,4,4,4,4,44,6]
print(len(e))
k = e[:5]
print(k)

u = e[3:7]
print(u)


25
[0, 1, 8, 3, 5]
[3, 5, 4, 5]
